# Aggergate osu random dumps into MongoDB
**Contributors:** Victor Lin

**Achievement:** Migrate osu_user_stats, osu_scores_high, osu_beatmaps, osu_beatmap_difficulty_attribs tables from several random data dumps into mongodb. Prioritize migrating more recent dumps to avoid duplicates. Created basic indecies on osu_scores_high & a unique compound index on osu_beatmap_attribs.

**Requirements:**

- External
    - MySQL DB has imported osu random dumps from https://data.ppy.sh/
- Other
    - env must be set with MySQL DB & Mongo log-in info

In [ ]:
import sys
sys.path.append('../..')
from exploration.config import sql_inst, mongo_inst
from mlpp.data_collection.sql_migration import SqlDumpMigrator

## Declare SQL dumps to migrate
**Note:** Order does matter in the declaration. More recent dumps should be declared first in order to prioritize more recent user information. Duplicate users found in older dumps will be skipped.

In [ ]:
# Order of insertion matters

SQL_DUMPS = [
    "osu_random_2021_01",
    "osu_random_2020_12",
    "osu_random_2020_11",
    "osu_random_2020_10",
    "osu_random_2020_09",
    "osu_random_2020_08"
]

migrator = SqlDumpMigrator(sql_inst)
osu_random_db = mongo_inst["osu_random_db"]

**TODO** : Code to create a compound index on (user_id, beatmap_id, and enabled_mods) would ensure score uniqueness. However, there are currently duplicates scores in some dumps that violates the uniqueness. Commented until further clarification from osu! devs.

In [ ]:
# mongo_db['osu_scores_high'].create_index(
#     [('user_id', pymongo.ASCENDING), ('beatmap_id', pymongo.ASCENDING) , ('enabled_mods', pymongo.ASCENDING)],
#     unique=True
# )

## Migrating Tables: Users, Scores, Beatmaps, Attribs

In [ ]:
osu_random_db['osu_scores_high'].create_index('user_id')
osu_random_db['osu_scores_high'].create_index('beatmap_id')

migrator.migrate_users_and_scores(SQL_DUMPS, osu_random_db)

In [ ]:
migrator.migrate_beatmaps(SQL_DUMPS, osu_random_db)

In [ ]:
migrator.migrate_beatmap_attribs(SQL_DUMPS, osu_random_db)